# Aluno: Felipe Gomes da Silva

# Exercícios de MapReduce
## 1. Utilizando o dataset flights.csv, faça operações MapReduce para calcular:
### a. O tempo total de vôo de cada companhia
### b. Qual é o destino mais visado
### c. Quais aeroportos cada companhia passou
### d. O vôo de maior distância de cada companhia
### e. Qual é o vôo mais frequente de cada companhia


# 2. Utilizando o dataset shakespeare.txt, faça uma operação de MapReduce para construir um índice invertido contendo como chave uma palavra e como valor uma lista de linhas onde ela foi mencionada. Exemplo:
### Entrada:
Aqui está um trecho

O trecho aqui

Está

Aqui

### Saída:
aqui [0, 1, 3]

está [0, 2]

o [1]

trecho [0, 1]

um [0]

### OBs: Rodando no Colab

In [1]:
pip install pyspark


     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 45.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=cce4c8c4c146895d35950e8601ddc517adb52f3df5d798f9b6708773578bc198
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.152 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u292-b10-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.152 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [3]:
import findspark
findspark.init('spark-3.2.0-bin-hadoop3.2')

import pyspark
import random


In [4]:
!git clone https://github.com/RafaelDaddio/BigDataAulasPUC.git

Cloning into 'BigDataAulasPUC'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 124 (delta 12), reused 21 (delta 8), pack-reused 94
Receiving objects: 100% (124/124), 18.85 MiB | 14.81 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [5]:
sc = pyspark.SparkContext(appName='Spark')

#1 - A - O tempo total de vôo de cada companhia


In [6]:
flights_RDD = sc.textFile('file:/content/BigDataAulasPUC/Datasets/Flights_old/flights.csv')
print(flights_RDD)

file:/content/BigDataAulasPUC/Datasets/Flights_old/flights.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [7]:
flights_header_RDD = sc.textFile('file:/content/BigDataAulasPUC/Datasets/Flights_old/flights_header.csv')
print(flights_header_RDD.collect())

['dataVoo,id_companhia,num_voos,origem,destino,hora_partida,min_atraso_partida,hora_chegada,min_atraso_chegada,duracao,distancia']


In [8]:
def read_line(line):
    record = line.split(',')
    return (record[1],float(record[9]))

tempo_voo_RDD = flights_RDD.map(read_line)
print(tempo_voo_RDD.reduceByKey(lambda a,b: a+b).take(10))

[('19930', 2001295.0), ('20409', 3009096.0), ('20366', 4142864.0), ('20437', 799733.0), ('19690', 449242.0), ('20355', 4194811.0), ('21171', 916098.0), ('19805', 6273366.0), ('19790', 7747256.0), ('20436', 689589.0)]


# 1 - B - Qual é o destino mais visado

In [9]:
def read_line(line):
    record = line.split(',')
    return (record[4], 1)

destino_RDD = flights_RDD.map(read_line)

destino_RDD = destino_RDD.reduceByKey(lambda a,b: a+b).collect()

outSort = sorted(destino_RDD, key=lambda b:b[1])

print('O destino mais visado é: ')
print(outSort[-1])


O destino mais visado é: 
('ATL', 30953)



# 1 - C - Quais aeroportos cada companhia passou

In [10]:
def read_line(line):
    record = line.split(',')
    return (record[1], record[3]+'-'+record[4])

aeroporto = flights_RDD.map(read_line).take(500000)
aeroporto_RDD = sc.parallelize(aeroporto)

sorted_grouped_RDD = aeroporto_RDD.groupByKey().sortByKey(ascending=True)
for key, values in sorted_grouped_RDD.collect():
    print(key, set(values))

19393 {'MCI-BWI', 'TUL-LAS', 'RSW-CLE', 'ONT-LAS', 'DAL-MSY', 'SAN-HOU', 'TUS-DEN', 'SAN-MCI', 'DEN-JAX', 'BWI-OKC', 'LAS-ICT', 'DEN-PDX', 'MSP-MDW', 'OAK-SNA', 'MDW-TUL', 'SNA-SFO', 'DAL-OKC', 'LBB-DAL', 'LAS-PDX', 'RSW-PHL', 'LAS-OKC', 'CMH-BNA', 'MDW-IAD', 'MEM-BWI', 'BUR-DEN', 'AUS-BNA', 'HOU-LIT', 'TPA-PVD', 'HOU-JAX', 'RSW-DTW', 'MKE-DEN', 'FLL-PHL', 'JAX-BWI', 'STL-MCO', 'TPA-PIT', 'ABQ-PDX', 'DEN-BWI', 'SFO-MKE', 'LIT-PHX', 'ATL-JAX', 'PHX-BWI', 'LAS-AUS', 'MKE-BOS', 'MDW-ONT', 'BNA-PHL', 'MDW-CHS', 'MSY-ATL', 'GEG-OAK', 'LAS-OAK', 'ABQ-OAK', 'SJC-BUR', 'LIT-HOU', 'OAK-BOI', 'ABQ-LAS', 'OKC-MDW', 'RSW-GRR', 'PHL-PHX', 'CHS-BNA', 'STL-CMH', 'ECP-STL', 'RSW-MDW', 'ATL-LAX', 'BUF-MCO', 'LAX-ABQ', 'TUL-MDW', 'LAX-SFO', 'CRP-HOU', 'FLL-MSY', 'MCO-FNT', 'MKE-LGA', 'MSY-BWI', 'ELP-HOU', 'HRL-AUS', 'BNA-JAX', 'BOS-DEN', 'MDW-MSY', 'OMA-LAS', 'DEN-ABQ', 'ISP-BWI', 'RDU-MCO', 'PHX-GEG', 'BNA-STL', 'PVD-TPA', 'FLL-LAS', 'MKE-MSP', 'DAY-MCO', 'OAK-GEG', 'MCI-RSW', 'OAK-SLC', 'SFO-SNA', 'PH

# 1 - D -  O vôo de maior distância de cada companhia

In [11]:
def read_line(line):
    record = line.split(',')
    return (record[1], float(record[10]))

destino_RDD = flights_RDD.map(read_line)

destino_RDD = destino_RDD.reduceByKey(lambda a,b: a if (a>b) else b).collect()

print(destino_RDD)

[('19930', 2874.0), ('20409', 2704.0), ('20366', 1389.0), ('20437', 2139.0), ('19690', 4983.0), ('20355', 2979.0), ('21171', 2704.0), ('19805', 3784.0), ('19790', 4502.0), ('20436', 1703.0), ('20398', 1379.0), ('20304', 1535.0), ('19977', 4962.0), ('19393', 2335.0)]


# 1 - E - Qual é o vôo mais frequente de cada companhia

In [12]:
voo = dict()
voo_count = dict()

def voos(line):
    record = line.split(',')
    return (record[1],record[3]+'-'+record[4])


mais_frequente = flights_RDD.map(voos).take(500000)
mais_frequente_RDD = sc.parallelize(mais_frequente)

grupo_mais_frequente_RDD = mais_frequente_RDD.groupByKey().sortByKey()

for frequente in grupo_mais_frequente_RDD.collect():
    voo[frequente[0]] = list(frequente[1])
    
for c in voo.keys():
    for id in voo[c]:
        if id in voo_count:
            voo_count[id] += 1
        else:
            voo_count[id] = 1
            
    lista =sorted(voo_count.values(),reverse=True)
    maximo = lista[0]
    id_max = list(voo_count.keys())[list(voo_count.values()).index(maximo)]
    voo_count.clear()
    
    print(c, '-', id_max)

19393 - DAL-HOU
19690 - OGG-HNL
19790 - LGA-ATL
19805 - DFW-LAX
19930 - LAX-SEA
19977 - SFO-ORD
20304 - SAN-LAX
20355 - BOS-DCA
20366 - ORD-CLE
20398 - ORD-CMH
20409 - MCO-JFK
20436 - DEN-LAS
20437 - DCA-ATL
21171 - SFO-LAX


# 2 - Utilizando o dataset shakespeare.txt, faça uma operação de MapReduce para construir um índice invertido contendo como chave uma palavra e como valor uma lista de linhas onde ela foi mencionada. 

In [23]:
shakespeare_file = sc.textFile('file:/content/BigDataAulasPUC/Datasets/shakespeare.txt')

In [24]:
print(shakespeare_file.take(100))

['hamlet@0\t\tHAMLET', 'hamlet@8\t', 'hamlet@9\t', 'hamlet@10\t\tDRAMATIS PERSONAE', 'hamlet@29\t', 'hamlet@30\t', 'hamlet@31\tCLAUDIUS\tking of Denmark. (KING CLAUDIUS:)', 'hamlet@74\t', 'hamlet@75\tHAMLET\tson to the late, and nephew to the present king.', 'hamlet@131\t', 'hamlet@132\tPOLONIUS\tlord chamberlain. (LORD POLONIUS:)', 'hamlet@176\t', 'hamlet@177\tHORATIO\tfriend to Hamlet.', 'hamlet@203\t', 'hamlet@204\tLAERTES\tson to Polonius.', 'hamlet@229\t', 'hamlet@230\tLUCIANUS\tnephew to the king.', 'hamlet@259\t', 'hamlet@260\t', 'hamlet@261\tVOLTIMAND\t|', 'hamlet@273\t\t|', 'hamlet@276\tCORNELIUS\t|', 'hamlet@288\t\t|', 'hamlet@291\tROSENCRANTZ\t|  courtiers.', 'hamlet@317\t\t|', 'hamlet@320\tGUILDENSTERN\t|', 'hamlet@335\t\t|', 'hamlet@338\tOSRIC\t|', 'hamlet@346\t', 'hamlet@347\t', 'hamlet@348\t\tA Gentleman, (Gentlemen:)', 'hamlet@375\t', 'hamlet@376\t\tA Priest. (First Priest:)', 'hamlet@403\t', 'hamlet@404\t', 'hamlet@405\tMARCELLUS\t|', 'hamlet@417\t\t|  officers.', 'ham

In [28]:
import nltk
import string

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [78]:

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

def read_line(line):
    record = line.split('@')
    resultado = record[1].split('\t')
    return (resultado[1], resultado[0])

def tokenizer(line):
    for token in nltk.word_tokenize(line):
        if token not in stopwords and token not in punctuation:
            yield token.lower()

shakespeare_RDD = sc.textFile('file:/content/BigDataAulasPUC/Datasets/shakespeare.txt')
tokens_RDD = shakespeare_RDD.map(read_line)

grouped_elements = tokens_RDD.groupByKey().collect()
lista = []
for key, values in grouped_elements:
  for i in nltk.word_tokenize(key):
        lista.append([i, list(values)])
print(lista)



[['CLAUDIUS', ['31', '788', '99918', '100020', '100099', '100292']], ['HORATIO', ['177', '1731', '2030', '2494', '2674', '3192', '3313', '3582', '3810', '3970', '4287', '4886', '6325', '7553', '7609', '7870', '8628', '16706', '16807', '17104', '17404', '17556', '17829', '17907', '18040', '18107', '18175', '18355', '19226', '19521', '19966', '20051', '20125', '20205', '20283', '20369', '20475', '20556', '20702', '27916', '27982', '28063', '28488', '29617', '30543', '30765', '30841', '31080', '31620', '31924', '32026', '32133', '37156', '37203', '37258', '37415', '37466', '37546', '37694', '37846', '38232', '38357', '38711', '38804', '38879', '39253', '39723', '86886', '87014', '88485', '97807', '97977', '98099', '98168', '129153', '138392', '138500', '138687', '138909', '152484', '153076', '153316', '154576', '154655', '158190', '158293', '158362', '158538', '161634', '163517', '163915', '164523', '164645', '165050', '165513', '165889', '166189', '166585', '167006', '167168', '169123', 